In [70]:
import pandas as pd
import requests, json
from datetime import date, timedelta, time
from collections import defaultdict
import xml.etree.ElementTree as ET
from math import dist

In [2]:
sample_df_humid = pd.read_csv("new_sample_humid.csv")
sample_df_weather = pd.read_csv("new_sample.csv")
wildfire_his = pd.read_csv("wildfire_his.csv")

In [10]:
# 먼저 2007년부터 2009년까지의 사건 발생 목록 추출
mask = wildfire_his['발생일시'] > 200701010000
temp = wildfire_his.loc[mask]
mask = wildfire_his['발생일시'] < 200912312359
small_his = temp.loc[mask]
small_his

,Unnamed: 0,_id,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
1564,1564,62be5e0852482b6dab132439,200912311525,목,200912311625,100,강원,강원,양양,현북,...,-3.1,0.0,30.0,0.0,0.0,6.1,,W,20,7.0
1565,1565,62be5e0852482b6dab13243a,200912301255,수,200912301355,100,경남,경남,김해,상동,...,5.5,0.0,39.1,0.0,0.0,7.8,,WNW,19,9.5
1566,1566,62be5e0852482b6dab13243b,200912301110,수,200912301145,35,울산,울산,울주,청량,...,7.4,0.0,39.6,0.0,0.0,6.2,,WSW,19,4.5
1567,1567,62be5e0852482b6dab13243c,200912291155,화,200912291510,315,울산,울산,울주,범서,...,3.4,0.0,34.4,0.0,0.0,0.8,,S,18,4.5
1568,1568,62be5e0852482b6dab13243d,200912281530,월,200912281620,50,경남,경남,양산,물금,...,3.8,0.0,33.5,0.0,0.0,3.3,,W,17,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2705,2705,62be5e0852482b6dab1328ae,200701120400,금,200701120600,200,경북,경북,칠곡,지천,...,-0.1,0.0,51.0,0.0,0.0,1.0,,NW,34,1.5
2706,2706,62be5e0852482b6dab1328af,200701081545,월,200701081750,205,경북,경북,영천,화북,...,4.1,0.0,30.0,0.0,0.0,4.6,,NW,30,4.5
2707,2707,62be5e0852482b6dab1328b0,200701072240,일,200701080850,1010,남부,경북,영천,자양,...,-1.0,0.0,64.0,0.0,0.0,4.2,,W,29,4.5
2708,2708,62be5e0852482b6dab1328b1,200701061640,토,200701061730,50,경북,경북,청도,각북,...,2.0,0.0,33.0,0.0,0.0,6.9,,W,24,4.0


In [17]:
mask = small_his['발생장소(시도)']=='강릉'
small_his[mask]

,Unnamed: 0,_id,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량


In [18]:
sample_df_weather

,Unnamed: 0,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,...,lcsCh,vs,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te
0,0,2007-01-01 00:00,1,90,속초,0.0,0,NaN,NaN,0.6,...,NaN,1500.0,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN
1,1,2007-01-01 01:00,2,90,속초,1.2,0,NaN,NaN,1.8,...,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN
2,2,2007-01-01 02:00,3,90,속초,2.0,0,NaN,NaN,2.1,...,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN
3,3,2007-01-01 03:00,4,90,속초,2.2,0,NaN,NaN,3.3,...,NaN,1200.0,0.0,NaN,-0.8,0,NaN,NaN,NaN,NaN
4,4,2007-01-01 04:00,5,90,속초,1.2,0,NaN,NaN,0.7,...,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60142,60142,2009-12-31 19:00,2204,216,태백,-12.3,0,NaN,NaN,2.8,...,NaN,NaN,NaN,NaN,-10.8,0,NaN,NaN,NaN,NaN
60143,60143,2009-12-31 20:00,2205,216,태백,-12.6,0,NaN,NaN,2.5,...,NaN,NaN,NaN,NaN,-11.4,0,NaN,NaN,NaN,NaN
60144,60144,2009-12-31 21:00,2206,216,태백,-12.8,0,NaN,NaN,1.8,...,NaN,NaN,NaN,NaN,-11.9,0,NaN,NaN,NaN,NaN
60145,60145,2009-12-31 22:00,2207,216,태백,-12.8,0,NaN,NaN,2.3,...,NaN,NaN,NaN,NaN,-12.2,0,NaN,NaN,NaN,NaN


In [85]:
sample_hm_merged=pd.merge(left=sample_df_weather, right=sample_df_humid, how='left', left_on=['tm','stnNm'], right_on=['일시','지점명'])

In [22]:
sample_hm_merged

,Unnamed: 0_x,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,...,tsQcflg,m005Te,m01Te,m02Te,m03Te,Unnamed: 0_y,지점번호,지점명,일시,실효습도
0,0,2007-01-01 00:00,1,90,속초,0.0,0,NaN,NaN,0.6,...,9,NaN,NaN,NaN,NaN,0.0,90.0,속초,2007-01-01 00:00,36.04
1,1,2007-01-01 01:00,2,90,속초,1.2,0,NaN,NaN,1.8,...,9,NaN,NaN,NaN,NaN,1.0,90.0,속초,2007-01-01 01:00,31.18
2,2,2007-01-01 02:00,3,90,속초,2.0,0,NaN,NaN,2.1,...,9,NaN,NaN,NaN,NaN,2.0,90.0,속초,2007-01-01 02:00,27.85
3,3,2007-01-01 03:00,4,90,속초,2.2,0,NaN,NaN,3.3,...,0,NaN,NaN,NaN,NaN,3.0,90.0,속초,2007-01-01 03:00,28.42
4,4,2007-01-01 04:00,5,90,속초,1.2,0,NaN,NaN,0.7,...,9,NaN,NaN,NaN,NaN,4.0,90.0,속초,2007-01-01 04:00,31.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60142,60142,2009-12-31 19:00,2204,216,태백,-12.3,0,NaN,NaN,2.8,...,0,NaN,NaN,NaN,NaN,60829.0,216.0,태백,2009-12-31 19:00,38.02
60143,60143,2009-12-31 20:00,2205,216,태백,-12.6,0,NaN,NaN,2.5,...,0,NaN,NaN,NaN,NaN,60830.0,216.0,태백,2009-12-31 20:00,37.86
60144,60144,2009-12-31 21:00,2206,216,태백,-12.8,0,NaN,NaN,1.8,...,0,NaN,NaN,NaN,NaN,60831.0,216.0,태백,2009-12-31 21:00,37.06
60145,60145,2009-12-31 22:00,2207,216,태백,-12.8,0,NaN,NaN,2.3,...,0,NaN,NaN,NaN,NaN,60832.0,216.0,태백,2009-12-31 22:00,37.22


In [24]:
# 기존 사건발생 데이터에서 지점번호 없는 것들은 가까운 지점으로 변경 후 사용
# 기존 사건발생 데이터에서 필요한것은 발생일시, 발생장소시도, 발생장소시군구 까지
occur_place_his = small_his[["발생일시","발생장소(시도)", "발생장소(시군구)"]]
occur_place_his

,발생일시,발생장소(시도),발생장소(시군구)
1564,200912311525,강원,양양
1565,200912301255,경남,김해
1566,200912301110,울산,울주
1567,200912291155,울산,울주
1568,200912281530,경남,양산
...,...,...,...
2705,200701120400,경북,칠곡
2706,200701081545,경북,영천
2707,200701072240,경북,영천
2708,200701061640,경북,청도


In [51]:
lat_df = pd.read_csv('latitude.csv')
lat_df

,Unnamed: 0,_id,지점번호,지점명,위도,경도
0,0,62be6fb4a82199005252108d,90,속초,38.25085,128.56473
1,1,62be6fb4a82199005252108e,93,북춘천,37.94738,127.75443
2,2,62be6fb4a82199005252108f,95,철원,38.14787,127.30420
3,3,62be6fb4a821990052521090,98,동두천,37.90188,127.06070
4,4,62be6fb4a821990052521091,99,파주,37.88589,126.76648
...,...,...,...,...,...,...
90,90,62be6fb4a8219900525210e7,285,합천,35.56505,128.16994
91,91,62be6fb4a8219900525210e8,288,밀양,35.49147,128.74413
92,92,62be6fb4a8219900525210e9,289,산청,35.41300,127.87910
93,93,62be6fb4a8219900525210ea,294,거제,34.88818,128.60458


In [27]:
# 지점번호 리스트에 없는 지점 위도 경도 찾기
KAKAO_API_KEY = "da33e829f9b3cce36b4dd72eac9312d6"
KAKAO_MAP_URL = "https://dapi.kakao.com/v2/local/search/address.json?query="
def get_lat(place):
    headers = {
        "Authorization" : "KakaoAK {}".format(KAKAO_API_KEY)
    }
    returned_json=json.loads(str(requests.get(KAKAO_MAP_URL+place,headers=headers).text))
    address = returned_json['documents'][0]['address']
    lat_d = {"lat" : str(address['y']), "lng" : str(address['x'])}
    return lat_d

In [29]:
# 중복제거
occur_place_his=occur_place_his.drop_duplicates()
occur_place_his

,발생일시,발생장소(시도),발생장소(시군구)
1564,200912311525,강원,양양
1565,200912301255,경남,김해
1566,200912301110,울산,울주
1567,200912291155,울산,울주
1568,200912281530,경남,양산
...,...,...,...
2705,200701120400,경북,칠곡
2706,200701081545,경북,영천
2707,200701072240,경북,영천
2708,200701061640,경북,청도


In [30]:
occur_place_his['지점명']=''
occur_place_his

,발생일시,발생장소(시도),발생장소(시군구),지점명
1564,200912311525,강원,양양,
1565,200912301255,경남,김해,
1566,200912301110,울산,울주,
1567,200912291155,울산,울주,
1568,200912281530,경남,양산,
...,...,...,...,...
2705,200701120400,경북,칠곡,
2706,200701081545,경북,영천,
2707,200701072240,경북,영천,
2708,200701061640,경북,청도,


In [52]:
# 지점번호 위도 경도 딕셔너리로 변환
point_dict={}
lat_df['지점명']=lat_df['지점명'].map(lambda x: x[:2] if len(x)>2 and (x[2]=='시' or x[2]=='군') else x)
lat_df=lat_df[["지점번호", "지점명","위도","경도"]]
for i in range(len(lat_df)):
    point_dict[lat_df.iloc[i][1]] = [lat_df.iloc[i][0], lat_df.iloc[i][2],lat_df.iloc[i][3]]
point_dict

{'속초': [90, 38.25085, 128.56473],
 '북춘천': [93, 37.94738, 127.75443],
 '철원': [95, 38.14787, 127.3042],
 '동두천': [98, 37.90188, 127.0607],
 '파주': [99, 37.88589, 126.76648],
 '대관령': [100, 37.67713, 128.71834],
 '춘천': [101, 37.90262, 127.7357],
 '백령도': [102, 37.97396, 124.71237],
 '북강릉': [104, 37.80456, 128.85535],
 '강릉': [105, 37.75147, 128.89099],
 '동해': [106, 37.50709, 129.12433],
 '서울': [108, 37.57142, 126.9658],
 '인천': [112, 37.47772, 126.6249],
 '원주': [114, 37.33749, 127.94659],
 '울릉도': [115, 37.48129, 130.89864],
 '수원': [119, 37.25746, 126.983],
 '영월': [121, 37.18126, 128.45743],
 '충주': [127, 36.97045, 127.9525],
 '서산': [129, 36.77658, 126.4939],
 '울진': [130, 36.99176, 129.41278],
 '청주': [131, 36.63924, 127.44066],
 '대전': [133, 36.37198, 127.37211],
 '추풍령': [135, 36.22025, 127.99458],
 '안동': [136, 36.57293, 128.70734],
 '상주': [137, 36.40837, 128.15741],
 '포항': [138, 36.03201, 129.38002],
 '군산': [140, 36.0053, 126.76135],
 '대구': [143, 35.87797, 128.65295],
 '전주': [146, 35.84092, 127.1

In [62]:
def close_or_exact_place(big_city, small_city):
    if big_city in point_dict.keys():
        return big_city
    elif small_city in point_dict.keys():
        return small_city
    else:
        try:
            point_lat=get_lat(big_city +' '+ small_city+'구')
            for_compare_distance = [] 
            for k in point_dict.keys():
                for_compare_distance.append([dist((float(point_lat['lat']),float(point_lat['lng'])),(point_dict[k][1],point_dict[k][2])), k])
            return sorted(for_compare_distance, key=lambda x: x[0])[0][1]
        except:
            try:
                point_lat=get_lat(big_city +' '+ small_city+'군')
                for_compare_distance = [] 
                for k in point_dict.keys():
                    for_compare_distance.append([dist((float(point_lat['lat']),float(point_lat['lng'])),(point_dict[k][1],point_dict[k][2])), k])
                
                return sorted(for_compare_distance, key=lambda x: x[0])[0][1]
            except:
                point_lat=get_lat(big_city +' '+ small_city+'시')
                for_compare_distance = [] 
                for k in point_dict.keys():
                    for_compare_distance.append([dist((float(point_lat['lat']),float(point_lat['lng'])),(point_dict[k][1],point_dict[k][2])), k])
                return sorted(for_compare_distance, key=lambda x: x[0])[0][1]

In [63]:
occur_place_his['지점명']=occur_place_his.apply(lambda x: close_or_exact_place(x['발생장소(시도)'], x['발생장소(시군구)']), axis=1)


In [64]:
# 가까운 지점 설정 완료
occur_place_his

,발생일시,발생장소(시도),발생장소(시군구),지점명
1564,200912311525,강원,양양,속초
1565,200912301255,경남,김해,김해
1566,200912301110,울산,울주,울산
1567,200912291155,울산,울주,울산
1568,200912281530,경남,양산,양산
...,...,...,...,...
2705,200701120400,경북,칠곡,구미
2706,200701081545,경북,영천,영천
2707,200701072240,경북,영천,영천
2708,200701061640,경북,청도,밀양


In [86]:
# 시간까지 발생시간 자르기
sample_hm_merged['tm'] = sample_hm_merged['tm'].map(lambda x:x[:-3].replace(' ',''))
sample_hm_merged['tm'] = sample_hm_merged['tm'].map(lambda x:x.replace('-',''))
#sample_hm_merged['label']=sample_hm_merged['tm'].map(lambda x:x[:-2])
sample_hm_merged

,Unnamed: 0_x,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,...,tsQcflg,m005Te,m01Te,m02Te,m03Te,Unnamed: 0_y,지점번호,지점명,일시,실효습도
0,0,2007010100,1,90,속초,0.0,0,NaN,NaN,0.6,...,9,NaN,NaN,NaN,NaN,0.0,90.0,속초,2007-01-01 00:00,36.04
1,1,2007010101,2,90,속초,1.2,0,NaN,NaN,1.8,...,9,NaN,NaN,NaN,NaN,1.0,90.0,속초,2007-01-01 01:00,31.18
2,2,2007010102,3,90,속초,2.0,0,NaN,NaN,2.1,...,9,NaN,NaN,NaN,NaN,2.0,90.0,속초,2007-01-01 02:00,27.85
3,3,2007010103,4,90,속초,2.2,0,NaN,NaN,3.3,...,0,NaN,NaN,NaN,NaN,3.0,90.0,속초,2007-01-01 03:00,28.42
4,4,2007010104,5,90,속초,1.2,0,NaN,NaN,0.7,...,9,NaN,NaN,NaN,NaN,4.0,90.0,속초,2007-01-01 04:00,31.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60142,60142,2009123119,2204,216,태백,-12.3,0,NaN,NaN,2.8,...,0,NaN,NaN,NaN,NaN,60829.0,216.0,태백,2009-12-31 19:00,38.02
60143,60143,2009123120,2205,216,태백,-12.6,0,NaN,NaN,2.5,...,0,NaN,NaN,NaN,NaN,60830.0,216.0,태백,2009-12-31 20:00,37.86
60144,60144,2009123121,2206,216,태백,-12.8,0,NaN,NaN,1.8,...,0,NaN,NaN,NaN,NaN,60831.0,216.0,태백,2009-12-31 21:00,37.06
60145,60145,2009123122,2207,216,태백,-12.8,0,NaN,NaN,2.3,...,0,NaN,NaN,NaN,NaN,60832.0,216.0,태백,2009-12-31 22:00,37.22


In [88]:
occur_place_his['발생일시'] = occur_place_his['발생일시'].map(lambda x : str(x)[:-2])
occur_place_his

,발생일시,발생장소(시도),발생장소(시군구),지점명
1564,2009123115,강원,양양,속초
1565,2009123012,경남,김해,김해
1566,2009123011,울산,울주,울산
1567,2009122911,울산,울주,울산
1568,2009122815,경남,양산,양산
...,...,...,...,...
2705,2007011204,경북,칠곡,구미
2706,2007010815,경북,영천,영천
2707,2007010722,경북,영천,영천
2708,2007010616,경북,청도,밀양


In [92]:
def occur_match(tm,name):
    for i in range(len(occur_place_his)):
        row = list(occur_place_his.iloc[i])
        if tm == row[0] and name == row[3]:
            return 1
        else:
            return 0

#라벨링
sample_hm_merged['label'] = sample_hm_merged.apply(lambda x: occur_match(x['tm'], x['stnNm']),axis=1)
sample_hm_merged

,Unnamed: 0_x,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,...,m005Te,m01Te,m02Te,m03Te,Unnamed: 0_y,지점번호,지점명,일시,실효습도,label
0,0,2007010100,1,90,속초,0.0,0,NaN,NaN,0.6,...,NaN,NaN,NaN,NaN,0.0,90.0,속초,2007-01-01 00:00,36.04,0
1,1,2007010101,2,90,속초,1.2,0,NaN,NaN,1.8,...,NaN,NaN,NaN,NaN,1.0,90.0,속초,2007-01-01 01:00,31.18,0
2,2,2007010102,3,90,속초,2.0,0,NaN,NaN,2.1,...,NaN,NaN,NaN,NaN,2.0,90.0,속초,2007-01-01 02:00,27.85,0
3,3,2007010103,4,90,속초,2.2,0,NaN,NaN,3.3,...,NaN,NaN,NaN,NaN,3.0,90.0,속초,2007-01-01 03:00,28.42,0
4,4,2007010104,5,90,속초,1.2,0,NaN,NaN,0.7,...,NaN,NaN,NaN,NaN,4.0,90.0,속초,2007-01-01 04:00,31.62,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60142,60142,2009123119,2204,216,태백,-12.3,0,NaN,NaN,2.8,...,NaN,NaN,NaN,NaN,60829.0,216.0,태백,2009-12-31 19:00,38.02,0
60143,60143,2009123120,2205,216,태백,-12.6,0,NaN,NaN,2.5,...,NaN,NaN,NaN,NaN,60830.0,216.0,태백,2009-12-31 20:00,37.86,0
60144,60144,2009123121,2206,216,태백,-12.8,0,NaN,NaN,1.8,...,NaN,NaN,NaN,NaN,60831.0,216.0,태백,2009-12-31 21:00,37.06,0
60145,60145,2009123122,2207,216,태백,-12.8,0,NaN,NaN,2.3,...,NaN,NaN,NaN,NaN,60832.0,216.0,태백,2009-12-31 22:00,37.22,0


In [96]:
final_sample_table=sample_hm_merged[["tm","stnNm",'ta','rn','ws','wd','hm','실효습도','label']]

In [97]:
final_sample_table.columns = ['일시','지점명','기온','강수량','풍속','풍향','습도','실효습도','label']
final_sample_table

,일시,지점명,기온,강수량,풍속,풍향,습도,실효습도,label
0,2007010100,속초,0.0,NaN,0.6,200.0,62,36.04,0
1,2007010101,속초,1.2,NaN,1.8,250.0,44,31.18,0
2,2007010102,속초,2.0,NaN,2.1,270.0,32,27.85,0
3,2007010103,속초,2.2,NaN,3.3,270.0,31,28.42,0
4,2007010104,속초,1.2,NaN,0.7,340.0,37,31.62,0
...,...,...,...,...,...,...,...,...,...
60142,2009123119,태백,-12.3,NaN,2.8,200.0,25,38.02,0
60143,2009123120,태백,-12.6,NaN,2.5,160.0,25,37.86,0
60144,2009123121,태백,-12.8,NaN,1.8,140.0,23,37.06,0
60145,2009123122,태백,-12.8,NaN,2.3,70.0,23,37.22,0


In [98]:
final_sample_table.to_csv('new_sample_with_humid_2007_2009.csv')